### TYK2 DMS/FlowDMS Global Rescaling

For BMS, we want to convert both the DMS and FlowDMS data to a 0-1 scale for interpretation individually, as well as for comparison on a "common" scale. For DMS, we use the normalized summary statistics from run 7, IFN-alpha 100 U/mL minus Untreated. For FlowDMS, we use the most recent midpoints computed from run 2. These formatted TSVs are located [here](../sumstats/BMS). We can make a simple plot of the relationship between the two derived scores across all variants:

In [2]:
library(ggpubr)
library(tidyverse)

In [3]:
midpoints <- read_tsv("/home/rstudio/bms-dms/sumstats/TYK2-VAMP/run2B/OCNT-VAMPLIB-1-assay-run2B-vampseq-midpoints.sumstats.tsv") %>%
    select(1:7) %>%
    mutate(mut_aa = if_else(mut_aa == "X", "*", mut_aa),
           chunk = as.numeric(gsub("rc", "", chunk)))

stop_scores <- midpoints %>%
    filter(mut_aa %in% c("X","*")) %>%
    group_by(chunk) %>%
    summarize("Stop score" = median(score_mean, na.rm = TRUE),
              "Stop score standard error" = median(score_se, na.rm = TRUE))

sample_stats_rescale <- midpoints %>%
    filter(mut_aa != "WT") %>%
    left_join(stop_scores, by = "chunk") %>%
    mutate(estimate = (score_mean - `Stop score`)/(`WT score` - `Stop score`),
           std.error = score_se/(`WT score` - `Stop score`))

flowdms_format <- sample_stats_rescale %>%
    select(chunk, pos, mut_aa, score_mean, score_se, estimate, std.error) %>%
    mutate("Midpoint_Lower" = score_mean - 2*score_se,
           "Midpoint_Upper" = score_mean - 2*score_se,
           "ScaledScore_Lower" = estimate - 2*std.error,
           "ScaledScore_Upper" = estimate + 2*std.error) %>%
    select(-std.error, -score_se) %>%
    rename("Chunk" = "chunk",
           "Position" = "pos",
           "AA" = "mut_aa",
           "Midpoint" = "score_mean",
           "ScaledScore" = "estimate") %>%
    relocate("ScaledScore", .after = "Midpoint_Upper") %>%
    arrange(Position, AA)

write_tsv(flowdms_format, "../sumstats/BMS/BMS-2/FlowDMS_midpoints_rescaled.tsv")

Rows: 23718 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (2): chunk, mut_aa
dbl (10): score_mean, score_se, pos, WT score, WT score standard error, esti...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
dms <- read_tsv("../sumstats/TYK2/run7/tyk2-assay7.tsv")  %>%
    mutate(aa = if_else(aa == "X", "*", aa),
           chunk = as.numeric(gsub("rc", "", chunk))) %>%
    filter(condition == "IFNalpha100_0 - None_0") %>%
    dplyr::select(chunk, pos, aa, estimate, std.error, statistic, p.value, p.adj) %>%
    mutate(fc = 2^estimate,
           fc_lci = 2^(estimate - 2*std.error),
           fc_uci = 2^(estimate + 2*std.error))

stop_scores_dms <- dms %>%
    filter(aa == "*") %>% 
    group_by(chunk) %>%
    summarize(stop_med = median(fc))

dms_rescale <- dms %>%
    left_join(stop_scores_dms, by = "chunk") %>%
    mutate(fc_rescale = (fc - stop_med)/(1 - stop_med),
           fc_lci_rescale = (fc_lci - stop_med)/(1 - stop_med),
           fc_uci_rescale = (fc_uci - stop_med)/(1 - stop_med))

dms_format <- dms_rescale %>%
    select(chunk, pos, aa, estimate, std.error, p.adj, fc_rescale, fc_lci_rescale, fc_uci_rescale) %>%
    rename("Chunk" = "chunk",
            "Position" = "pos",
            "AA" = "aa",
            "Log2FoldChange" = "estimate",
            "Log2FoldChangeError" = "std.error",
            "FDR" = "p.adj",
            "ScaledScore" = "fc_rescale",
            "ScaledScore_Lower" = "fc_lci_rescale",
            "ScaledScore_Upper" = "fc_uci_rescale") %>%
    arrange(Position, AA)


write_tsv(dms_format, "../sumstats/BMS/BMS-2/DMS_IFNalpha100vsUntreated_rescaled.tsv")

Rows: 94876 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): drug, clone, chunk, aa, version, condition
dbl (8): conc, pos, estimate, std.error, dispersion, statistic, p.value, p.adj

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
inner_join(dms_format, flowdms_format, by = join_by(Chunk, Position, AA)) %>%
    ggplot() +
        geom_point(aes(x = ScaledScore.x,
                       y = ScaledScore.y,
                       color = if_else(AA == "*", "red", "black"))) +
        coord_cartesian(xlim = c(-0.5, 2),
                        ylim = c(-0.5, 1.5)) +
        theme_pubr(base_size = 20) +
        xlab("DMS Score") +
        ylab("FlowDMS Score") +
        scale_color_identity() +
        ggtitle("TYK2 Global DMS::FlowDMS Rescaling")